In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoConfig
import torch
from customModel import customBertForSequenceClassification, customRobertaForSequenceClassification,customGPT2ForSequenceClassification, customElectraForSequenceClassification
import numpy as np
from frontModelCustom import id2labelData_labels


/home/leesk/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Model_path = './best_model/e_60_t_40_g_52'
# Model_path = './my_awesome_model/checkpoint-5100'
# Model_path = 'SangGank/my-front-model'
tokenizer = AutoTokenizer.from_pretrained(Model_path)
model = customRobertaForSequenceClassification.from_pretrained(Model_path)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


In [3]:
import pandas as pd

In [4]:
data = pd.read_csv("./genre_11_tempo_4.csv")

In [5]:
data2 = data.copy()
emotion_data = data2.groupby('emotion').sample(frac=0.05, random_state=42)
tempo_data = data2.groupby('tempo(category)').sample(frac=0.05, random_state=42)
genre_data = data2.groupby('genre').sample(frac=0.05, random_state=42)
index_total = set(emotion_data.index) | set(tempo_data.index) | set(genre_data.index)
valid_data = data2.iloc[list(index_total)]
train_data = data2.drop(list(index_total)).sample(frac=1, random_state=42)

In [6]:
valid_data.emotion.unique()

array(['excitement', 'nostalgia', 'anger', 'love', 'sadness', 'happiness',
       'loneliness', 'anticipation', 'calmness', 'gratitude'],
      dtype=object)

In [7]:
valid_data[valid_data['emotion']== 'nostalgia']['caption'].values

array(["The song's melody is a soothing and uplifting blend of piano and guitar chords, with a catchy and memorable hook that lingers in the listener's mind.",
       'emotive and haunting, with a soaring vocal performance by Ann Wilson that showcases her signature range and power.',
       'a soothing, gentle flow with a catchy and memorable chorus.',
       "catchy and upbeat, with a memorable and sing-along chorus that showcases the band's ability to craft infectious and memorable pop hooks.",
       'soulful and bluesy, with a repetitive and catchy bassline that drives the rhythm and creates a sense of relaxed, easygoing atmosphere.',
       'smooth and sultry, with a gentle, lilting quality that evokes a sense of longing and nostalgia.',
       "The song's melody is catchy and upbeat, with a memorable chorus and a mix of guitar and synthesizer instrumentation that creates a lively and energetic sound.",
       'catchy and upbeat, with a memorable chorus and a distinctive guitar ri

In [8]:
# for i in valid_data['caption'].values:
#     inputs = tokenizer(i, return_tensors='pt')
#     result = model(**inputs).logits
#     emo, tem, gen = logits2Label(result)
#     if gen != 'Rock':
#         print(i)

In [9]:
model.config

BertConfig {
  "_name_or_path": "./best_model/e_60_t_40_g_52",
  "architectures": [
    "customRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels1": 10,
  "num_labels2": 4,
  "num_labels3": 11,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "multi_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.38.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [10]:
text= 'smooth and sultry, with a catchy and memorable piano riff that sets the tone for the rest of the track'
text= 'very sad song give me'
inputs = tokenizer(text, return_tensors='pt')
print(inputs)

result = model(**inputs).logits
print(result)

{'input_ids': tensor([[    0,  9426, 22571,     8,   579,  6070,  1506,     6,    19,    10,
         33200,     8, 10132, 13305, 31457,    14,  3880,     5,  6328,    13,
             5,  1079,     9,     5,  1349,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}
pooled_output :  tensor([[ 0.8187, -0.3696,  0.0481,  ..., -1.0249, -0.4952, -0.1013]],
       grad_fn=<SliceBackward0>)
logits1 :  tensor([[-1.3488, -0.1706, -1.8826, -3.0804, -3.0152, -3.8921, -3.3904, -6.1804,
         -6.1386, -6.3835]], grad_fn=<AddmmBackward0>)
logits2 :  tensor([[-0.6902, -0.5398, -1.9985, -1.8040]], grad_fn=<AddmmBackward0>)
logits3 :  tensor([[-0.7906, -0.8887, -2.7787, -3.2877, -2.5192, -3.7893, -2.6526, -3.4286,
         -4.7993, -3.6399, -4.5625]], grad_fn=<AddmmBackward0>)
(tensor([[-1.3488, -0.1706, -1.8826, -3.0804, -3.0152, -3.8921, -3.3904, -6.1804,
         -6.1386, -6.3835]], grad_fn=<AddmmBackward0>), tensor([[-0

In [9]:
model.classifier1.weight

Parameter containing:
tensor([[-0.0132,  0.0145,  0.0467,  ...,  0.0158, -0.0164,  0.0089],
        [-0.0348,  0.0241, -0.0217,  ...,  0.0371, -0.0127, -0.0233],
        [ 0.0001, -0.0322,  0.0084,  ..., -0.0037, -0.0152,  0.0145],
        ...,
        [ 0.0020,  0.0227,  0.0166,  ...,  0.0063, -0.0023,  0.0013],
        [-0.0132, -0.0187,  0.0003,  ..., -0.0009, -0.0028, -0.0308],
        [ 0.0332, -0.0090, -0.0171,  ..., -0.0037, -0.0349, -0.0501]],
       requires_grad=True)

In [72]:
import pickle
label_data_path ='./labels.pkl'
with open(label_data_path,'rb') as f:
        emotion_labels=pickle.load(f)
        tempo_labels=pickle.load(f)
        genre_labels=pickle.load(f)

print(tempo_labels)

['Allegro' 'Moderato' 'Andante' 'Presto']


In [73]:
emotion , tempo, genre = id2labelData_labels('labels.pkl')

def logits2Label(logits):
    print(logits)
    emotion_id = int(logits[0].detach().argmax(),dim=1)
    tempo_id = int(logits[1].detach().argmax(),dim=1)
    genre_id = int(logits[2].detach().argmax(),dim=1)
    return emotion[emotion_id], tempo[tempo_id], genre[genre_id]

In [74]:
emotion[1], tempo[1], genre[0]

('excitement', 'Moderato', 'Rock')

In [75]:
logits2Label(result)

(tensor([[-1.3488, -0.1706, -1.8826, -3.0804, -3.0152, -3.8921, -3.3904, -6.1804,
         -6.1386, -6.3835]], grad_fn=<AddmmBackward0>), tensor([[-0.6902, -0.5398, -1.9985, -1.8040]], grad_fn=<AddmmBackward0>), tensor([[-0.7906, -0.8887, -2.7787, -3.2877, -2.5192, -3.7893, -2.6526, -3.4286,
         -4.7993, -3.6399, -4.5625]], grad_fn=<AddmmBackward0>))


TypeError: 'dim' is an invalid keyword argument for int()

In [ ]:
import torch

print(torch.__version__)

2.2.1+cu121
